In [63]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import os
import requests
from tqdm import tqdm
import json

print(os.getcwd())

import sys
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

from meteostat import Point, Hourly
import pandas as pd
from datetime import datetime

/Users/tymoteuszbarcinski/DTU/WEC2025/wec2025/TB


In [11]:
Path().resolve().parent

# Add utils to path
ROOT_PATH = Path().resolve().parent
print(f"{ROOT_PATH=}")
DATA_PATH = ROOT_PATH / "for_participants" / "data_parquet"
# assert DATA_PATH.exists()
sys.path.append(str(ROOT_PATH))
print(f"{DATA_PATH=}")

# ASSIGNMENT_DIR = Path().resolve().parent
# HANDIN_DIR = ASSIGNMENT_DIR / 'handin'
OUTPUT_DIR = ROOT_PATH / 'output'

def read_data(file_name):
    df_tmp = pd.read_parquet(DATA_PATH / f'{file_name}.parquet')
    print(df_tmp.shape)
    return df_tmp

ROOT_PATH=PosixPath('/Users/tymoteuszbarcinski/DTU/WEC2025/wec2025')
DATA_PATH=PosixPath('/Users/tymoteuszbarcinski/DTU/WEC2025/wec2025/for_participants/data_parquet')


# Stations

In [17]:
df_traffic_train = read_data('traffic_train')
df_traffic_test = read_data('traffic_test_without_target')

(102480, 9)
(67344, 8)


In [32]:
df_stations_train = df_traffic_train.drop_duplicates(subset=['station_id'], keep='first')
df_stations_train['type'] = "train"
df_traffic_test = df_traffic_test.drop_duplicates(subset=['station_id'], keep='first')
df_traffic_test['type'] = "test"

date_min = df_traffic_train['date'].sort_values().min()
date_max = df_traffic_train['date'].sort_values().max()

df_stations = pd.concat([df_stations_train, df_traffic_test], axis=0)

In [42]:
lon, lat = df_stations.iloc[0, :]['lon'], df_stations.iloc[0, :]['lat']
print(f"lon: {lon}, lat: {lat}")

lon: 20.9321, lat: 52.165


# MeteoStat

In [67]:
start = datetime.strptime(f"{str(date_min)} 00:00:00", "%Y-%m-%d %H:%M:%S")
end = datetime.strptime(f"{str(date_max)} 23:59:59", "%Y-%m-%d %H:%M:%S")

df_meteo  = pd.DataFrame()

for _, row in tqdm(df_stations.iterrows()):

    warsaw = Point(lat = row['lat'], lon = row['lon'])
    data = Hourly(warsaw, start, end)
    data = data.fetch()
    data['lon'] = row['lat']
    data['lat'] = row['lon']
    data['station_id'] = row['station_id']

    df_meteo = pd.concat([df_meteo, data], axis=0)

    # Display the data
    # print(data[["temp", "prcp", "cldc"]])  # Temperature, precipitation, cloud cover

116it [00:02, 43.42it/s]


In [73]:
df_meteo.loc['2024-03-01 00:00:00']['temp'].unique()

array([9.4])

In [77]:
df_meteo.reset_index().groupby('time').std()

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,lon,lat,station_id
time,,,,,,,,,,,,,,
2024-03-01 00:00:00,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.048213,0.077458,1679.160028
2024-03-01 01:00:00,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.048213,0.077458,1679.160028
2024-03-01 02:00:00,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.048213,0.077458,1679.160028
2024-03-01 03:00:00,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.048213,0.077458,1679.160028
2024-03-01 04:00:00,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.048213,0.077458,1679.160028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-30 19:00:00,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.048213,0.077458,1679.160028
2024-04-30 20:00:00,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.048213,0.077458,1679.160028
2024-04-30 21:00:00,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.048213,0.077458,1679.160028


In [46]:
data.rename(columns={
    "temp": "temperature",
    "dwpt": "dew_point_temperature",
    "rhum": "relative_humidity",
    "prcp": "precipitation",
    "snow": "snowfall",
    "wdir": "wind_direction",
    "wspd": "wind_speed",
    "wpgt": "wind_gust",
    "pres": "pressure",
    "tsun": "sunshine_duration",
    "coco": "weather_condition_code"
}, inplace=True)

# Open Meteo

In [105]:
start = datetime.strptime(f"{str(date_min)} 00:00:00", "%Y-%m-%d %H:%M:%S")
end = datetime.strptime(f"{str(date_max)} 23:59:59", "%Y-%m-%d %H:%M:%S")

# Define the API endpoint and parameters
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 52.2297,
    "longitude": 21.0122,
    "start_date": str(date_min),
    "end_date": str(date_max),
    "hourly": "temperature_2m,dewpoint_2m,relative_humidity_2m,precipitation,snowfall,cloudcover,shortwave_radiation,windspeed_10m,winddirection_10m,pressure_msl",
    "timezone": "Europe/Warsaw"
}

df_meteo  = pd.DataFrame()

for _, row in tqdm(df_stations.iterrows()):

    params['latitude'] = row['lat']
    params['longitude'] = row['lon']
    # Fetch cloud cover data
    response = requests.get(url, params=params)
    data = response.json()

    data =  pd.DataFrame(data['hourly'])

    data['lon'] = row['lat']
    data['lat'] = row['lon']
    data['station_id'] = row['station_id']
    data['h3res13'] = row['h3res13']
    
    df_meteo = pd.concat([df_meteo, data], axis=0)

116it [00:32,  3.55it/s]


In [107]:
df_meteo.to_parquet(DATA_PATH / 'meteo.parquet', index=False)